### Лабораторная работа: Регрессия

В сегодняшней лабораторной будем работать с данными о проданных на аукционе автомобилях (`Carvana Car Prediction`). Целевым признаком здесь является наличие скрытых продавцом существенных недостатков. 

Carvana &mdash; компания-перекупщик автомобилей, соответственно, ее задача &mdash; не допустить скупку автомобилей со скрытыми недостатками, так как их впоследствии тяжелее продавать.

## 1

Загрузите данные о проданных на аукционе автомобилях. Посмотрите на "сырые" данные. 

Просмотрите файл `DataDictionary-ru.txt` и поймите, что означают столбцы матрицы. Выведите статистику по столбцам.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
# место для кода
df = pd.read_csv('data.csv')
df.drop(['RefId'], axis = 1, inplace=True)
df.head(10)

,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,Color,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,RED,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,WHITE,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,MAROON,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,0,12/7/2009,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,SILVER,...,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,SILVER,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020
5,0,12/7/2009,ADESA,2004,5,MITSUBISHI,GALANT 4C,ES,4D SEDAN ES,WHITE,...,8149.0,9451.0,NaN,NaN,19638,33619,FL,5600.0,0,594
6,0,12/7/2009,ADESA,2004,5,KIA,SPECTRA,EX,4D SEDAN EX,BLACK,...,6230.0,8603.0,NaN,NaN,19638,33619,FL,4200.0,0,533
7,0,12/7/2009,ADESA,2005,4,FORD,TAURUS,SE,4D SEDAN SE,WHITE,...,6942.0,8242.0,NaN,NaN,19638,33619,FL,4500.0,0,825
8,0,12/7/2009,ADESA,2007,2,KIA,SPECTRA,EX,4D SEDAN EX,BLACK,...,9637.0,10778.0,NaN,NaN,21973,33619,FL,5600.0,0,482
9,0,12/7/2009,ADESA,2007,2,FORD,FIVE HUNDRED,SEL,4D SEDAN SEL,RED,...,12580.0,14845.0,NaN,NaN,21973,33619,FL,7700.0,0,1633


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 33 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   IsBadBuy                           72983 non-null  int64  
 1   PurchDate                          72983 non-null  object 
 2   Auction                            72983 non-null  object 
 3   VehYear                            72983 non-null  int64  
 4   VehicleAge                         72983 non-null  int64  
 5   Make                               72983 non-null  object 
 6   Model                              72983 non-null  object 
 7   Trim                               70623 non-null  object 
 8   SubModel                           72975 non-null  object 
 9   Color                              72975 non-null  object 
 10  Transmission                       72974 non-null  object 
 11  WheelTypeID                        69814 non-null  flo

In [4]:
df.drop(['Model', 'Trim', 'SubModel'],axis=1,inplace=True)

In [5]:
df['Transmission'].value_counts(dropna=False)

AUTO      70398
MANUAL     2575
NaN           9
Manual        1
Name: Transmission, dtype: int64

In [6]:
df["Transmission"].replace("Manual","MANUAL",inplace=True)

In [7]:
df["Transmission"].fillna(value="unknown",inplace=True)

In [8]:
df.drop('WheelTypeID',axis=1,inplace=True)

In [9]:
df['WheelType'].value_counts(dropna=False)

Alloy      36050
Covers     33004
NaN         3174
Special      755
Name: WheelType, dtype: int64

In [10]:
df["WheelType"].fillna(value="unknown",inplace=True)

In [11]:
df['PRIMEUNIT'].value_counts(dropna=False)

NaN    69564
NO      3357
YES       62
Name: PRIMEUNIT, dtype: int64

In [12]:
df["PRIMEUNIT"].fillna(value="unknown",inplace=True)

In [13]:
df[df['IsBadBuy']==1]['AUCGUART'].value_counts(dropna=False)

NaN      8843
GREEN     125
RED         8
Name: AUCGUART, dtype: int64

In [14]:
df["AUCGUART"].fillna(value="unknown",inplace=True)

In [15]:
df[df['IsBadBuy']==1]['IsOnlineSale'].value_counts(dropna=False)

0    8763
1     213
Name: IsOnlineSale, dtype: int64

In [16]:
df[df['IsBadBuy']==0]['IsOnlineSale'].value_counts(dropna=False)

0    62375
1     1632
Name: IsOnlineSale, dtype: int64

In [17]:
df[df['IsBadBuy']==1]['VNZIP1'].value_counts(dropna=False)

32824    347
75236    336
27542    330
29697    320
85226    309
        ... 
37138      1
77301      1
48265      1
64153      1
23234      1
Name: VNZIP1, Length: 131, dtype: int64

In [18]:
df[df['IsBadBuy']==1]['MMRAcquisitionAuctionAveragePrice'].value_counts()

0.0       114
7811.0     12
2765.0     11
4573.0     11
5427.0     10
         ... 
2022.0      1
7058.0      1
5700.0      1
3930.0      1
1996.0      1
Name: MMRAcquisitionAuctionAveragePrice, Length: 5148, dtype: int64

In [19]:
df['WarrantyCost'].value_counts(dropna=False)

920     2870
1974    2407
2152    2082
1389    2078
1215    2006
        ... 
683        1
2647       1
2538       1
2815       1
7198       1
Name: WarrantyCost, Length: 281, dtype: int64

In [20]:
df[df['IsBadBuy']==0]['MMRAcquisitionAuctionAveragePrice'].value_counts()

0.0        714
5480.0     360
5569.0     226
6311.0     167
6858.0     132
          ... 
5495.0       1
10592.0      1
15925.0      1
4313.0       1
9842.0       1
Name: MMRAcquisitionAuctionAveragePrice, Length: 10050, dtype: int64

In [18]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,IsBadBuy,VehYear,VehicleAge,WheelTypeID,VehOdo,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,BYRNO,VNZIP1,VehBCost,IsOnlineSale,WarrantyCost
IsBadBuy,1.000000,-0.158886,0.167164,-0.044620,0.082560,-0.109252,-0.102954,-0.087418,-0.083665,-0.109112,-0.104020,-0.103914,-0.100245,-0.061488,0.005796,-0.099911,-0.003697,0.052319
VehYear,-0.158886,1.000000,-0.957957,0.259660,-0.283719,0.581146,0.532927,0.581191,0.539230,0.590820,0.548426,0.598490,0.558669,0.280791,0.068276,0.349965,0.057380,-0.268696
VehicleAge,0.167164,-0.957957,1.000000,-0.250801,0.319794,-0.565690,-0.518810,-0.460854,-0.426382,-0.575607,-0.532505,-0.504062,-0.469359,-0.268269,-0.074857,-0.312329,-0.024771,0.259310
WheelTypeID,-0.044620,0.259660,-0.250801,1.000000,-0.213558,-0.095471,-0.129597,-0.072700,-0.102019,-0.088597,-0.121211,-0.074420,-0.102488,0.187199,0.007901,-0.162826,-0.006747,-0.132798
VehOdo,0.082560,-0.283719,0.319794,-0.213558,1.000000,-0.017022,0.024617,0.031306,0.063165,-0.028568,0.012107,0.015811,0.048005,-0.288456,-0.053041,-0.062008,0.029556,0.411718
MMRAcquisitionAuctionAveragePrice,-0.109252,0.581146,-0.565690,-0.095471,-0.017022,1.000000,0.990347,0.910391,0.910187,0.937817,0.932492,0.871755,0.871042,0.107734,0.045694,0.789840,0.038818,-0.050417
MMRAcquisitionAuctionCleanPrice,-0.102954,0.532927,-0.518810,-0.129597,0.024617,0.990347,1.000000,0.902718,0.918242,0.923610,0.932114,0.860716,0.871203,0.062502,0.040675,0.781516,0.038667,-0.020644
MMRAcquisitionRetailAveragePrice,-0.087418,0.581191,-0.460854,-0.072700,0.031306,0.910391,0.902718,1.000000,0.990201,0.851647,0.849407,0.913108,0.906054,0.107516,0.035489,0.746126,0.080627,-0.053444
MMRAcquisitonRetailCleanPrice,-0.083665,0.539230,-0.426382,-0.102019,0.063165,0.910187,0.918242,0.990201,1.000000,0.846914,0.855689,0.903188,0.906810,0.069676,0.031284,0.744966,0.077670,-0.026933
MMRCurrentAuctionAveragePrice,-0.109112,0.590820,-0.575607,-0.088597,-0.028568,0.937817,0.923610,0.851647,0.846914,1.000000,0.990329,0.915073,0.912231,0.111363,0.048965,0.778738,0.039876,-0.056798


# Регрессия

In [19]:
X = df[['VehicleAge', 'Make', 'Transmission', 'WheelType', 'VehOdo', 'VehBCost', 'VNZIP1', 'WarrantyCost', 'AUCGUART', 'PRIMEUNIT']]
y = df['IsBadBuy']

In [20]:
X.head()

,VehicleAge,Make,Transmission,WheelType,VehOdo,VehBCost,VNZIP1,WarrantyCost,AUCGUART,PRIMEUNIT
0,3,MAZDA,AUTO,Alloy,89046,7100.0,33619,1113,NaN,NaN
1,5,DODGE,AUTO,Alloy,93593,7600.0,33619,1053,NaN,NaN
2,4,DODGE,AUTO,Covers,73807,4900.0,33619,1389,NaN,NaN
3,5,DODGE,AUTO,Alloy,65617,4100.0,33619,630,NaN,NaN
4,4,FORD,MANUAL,Covers,69367,4000.0,33619,1020,NaN,NaN


# Категоризуем нечисловые данные

In [21]:
X = pd.get_dummies(X, columns=["Make"])

In [22]:
X = pd.get_dummies(X, columns=["Transmission"])
X = pd.get_dummies(X, columns=["WheelType"])
X = pd.get_dummies(X, columns=["PRIMEUNIT"])
X = pd.get_dummies(X, columns=["AUCGUART"])

In [29]:
X

,VehicleAge,VehOdo,VehBCost,VNZIP1,WarrantyCost,Make_ACURA,Make_BUICK,Make_CADILLAC,Make_CHEVROLET,Make_CHRYSLER,...,Transmission_AUTO,Transmission_MANUAL,Transmission_Manual,WheelType_Alloy,WheelType_Covers,WheelType_Special,PRIMEUNIT_NO,PRIMEUNIT_YES,AUCGUART_GREEN,AUCGUART_RED
0,0.333333,0.769515,0.156150,0.338819,0.148440,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
1,0.555556,0.808809,0.167147,0.338819,0.140437,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
2,0.444444,0.637823,0.107766,0.338819,0.185249,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,0.555556,0.567047,0.090171,0.338819,0.084022,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,0.444444,0.599454,0.087972,0.338819,0.136036,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72978,0.888889,0.390902,0.092371,0.304483,0.132435,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
72979,0.222222,0.620125,0.136357,0.304483,0.138437,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
72980,0.444444,0.764797,0.180343,0.304483,0.252467,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
72981,0.333333,0.687488,0.153951,0.304483,0.263270,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0


In [23]:
X.shape

(72983, 48)

нормируем данные

In [24]:
for i in ['VehicleAge', 'VehOdo', 'VehBCost', 'VNZIP1', 'WarrantyCost']:
    max_val = max(X[i])
    X[i] = X[i] / max_val

In [25]:
X

,VehicleAge,VehOdo,VehBCost,VNZIP1,WarrantyCost,Make_ACURA,Make_BUICK,Make_CADILLAC,Make_CHEVROLET,Make_CHRYSLER,...,Transmission_AUTO,Transmission_MANUAL,Transmission_Manual,WheelType_Alloy,WheelType_Covers,WheelType_Special,PRIMEUNIT_NO,PRIMEUNIT_YES,AUCGUART_GREEN,AUCGUART_RED
0,0.333333,0.769515,0.156150,0.338819,0.148440,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
1,0.555556,0.808809,0.167147,0.338819,0.140437,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
2,0.444444,0.637823,0.107766,0.338819,0.185249,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,0.555556,0.567047,0.090171,0.338819,0.084022,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,0.444444,0.599454,0.087972,0.338819,0.136036,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72978,0.888889,0.390902,0.092371,0.304483,0.132435,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
72979,0.222222,0.620125,0.136357,0.304483,0.138437,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
72980,0.444444,0.764797,0.180343,0.304483,0.252467,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
72981,0.333333,0.687488,0.153951,0.304483,0.263270,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0


## 3a

Импортируйте из библиотеки `sklearn` все модели машинного обучения:

`from sklearn import *`

Трактуя задачу о предсказании наличия недостатков как задачу регрессии, натренируйте линейную модель [`LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) на каком-нибудь подмножестве признаков (например, тех же `VehOdo` и `MMRCurrentRetailAveragePrice`). 

С какой проблемой вы столкнулись? Какие способы ее решения вы можете предложить?

*Функции, которые могут пригодиться при решении: `.fit(), .loc[], pd.notnull()`*

*Создание модели:* `model1 = linear_model.LinearRegression()`

*Тренировка:*
`model1.fit(x, y)`



In [26]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
pca.fit(X)
#X = pca.transform(X)

PCA(n_components=5)

In [27]:
X

,VehicleAge,VehOdo,VehBCost,VNZIP1,WarrantyCost,Make_ACURA,Make_BUICK,Make_CADILLAC,Make_CHEVROLET,Make_CHRYSLER,...,Transmission_AUTO,Transmission_MANUAL,Transmission_Manual,WheelType_Alloy,WheelType_Covers,WheelType_Special,PRIMEUNIT_NO,PRIMEUNIT_YES,AUCGUART_GREEN,AUCGUART_RED
0,0.333333,0.769515,0.156150,0.338819,0.148440,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
1,0.555556,0.808809,0.167147,0.338819,0.140437,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
2,0.444444,0.637823,0.107766,0.338819,0.185249,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,0.555556,0.567047,0.090171,0.338819,0.084022,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,0.444444,0.599454,0.087972,0.338819,0.136036,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72978,0.888889,0.390902,0.092371,0.304483,0.132435,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
72979,0.222222,0.620125,0.136357,0.304483,0.138437,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
72980,0.444444,0.764797,0.180343,0.304483,0.252467,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
72981,0.333333,0.687488,0.153951,0.304483,0.263270,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0


In [28]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 48 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   VehicleAge           72983 non-null  float64
 1   VehOdo               72983 non-null  float64
 2   VehBCost             72983 non-null  float64
 3   VNZIP1               72983 non-null  float64
 4   WarrantyCost         72983 non-null  float64
 5   Make_ACURA           72983 non-null  uint8  
 6   Make_BUICK           72983 non-null  uint8  
 7   Make_CADILLAC        72983 non-null  uint8  
 8   Make_CHEVROLET       72983 non-null  uint8  
 9   Make_CHRYSLER        72983 non-null  uint8  
 10  Make_DODGE           72983 non-null  uint8  
 11  Make_FORD            72983 non-null  uint8  
 12  Make_GMC             72983 non-null  uint8  
 13  Make_HONDA           72983 non-null  uint8  
 14  Make_HUMMER          72983 non-null  uint8  
 15  Make_HYUNDAI         72983 non-null 

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [31]:
X_train.shape

(51088, 48)

In [33]:
# место для кода
from sklearn import *
model_LR = linear_model.LinearRegression()
model_LR.fit(X_train, y_train)
model_Ridge = linear_model.Ridge()
model_Ridge.fit(X_train, y_train)
model_Lasso = linear_model.Lasso()
model_Lasso.fit(X_train, y_train)
model_SVM = svm.SVR()
model_SVM.fit(X_train, y_train)



SVR()

In [32]:
X_test.shape

(21895, 48)

In [34]:
model_SVM_L = svm.SVR(kernel='linear')
model_SVM_L.fit(X_train, y_train)


SVR(kernel='linear')

In [36]:
model_LR = linear_model.LinearRegression()
model_LR.fit(X_train, y_train)

LinearRegression()

## 3b

Выполните предсказание для всех объектов обучающей выборки и присвойте результат переменной prediction

*Функции, которые могут пригодиться при решении: `model.predict()`*

In [37]:
# место для кода
y_pred_LR = model_LR.predict(X_test)
y_pred_Ridge = model_Ridge.predict(X_test)
y_pred_Lasso = model_Lasso.predict(X_test)
y_pred_SVM = model_SVM.predict(X_test)
y_pred_SVM_L = model_SVM_L.predict(X_test)

In [38]:
y_pred_LR

array([ 0.17430115,  0.16656733,  0.08031607, ..., -0.01309776,
        0.08654404,  0.1167078 ])

In [39]:
y_pred_Ridge

array([ 0.17410008,  0.16761752,  0.08070895, ..., -0.01302859,
        0.08630534,  0.11671339])

In [40]:
y_pred_Lasso

array([0.12317961, 0.12317961, 0.12317961, ..., 0.12317961, 0.12317961,
       0.12317961])

In [41]:
y_pred_SVM

array([0.10015808, 0.09660201, 0.09980584, ..., 0.09215184, 0.09982252,
       0.09970103])

In [42]:
y_pred_SVM_L

array([0.09999618, 0.09988127, 0.10015095, ..., 0.10006537, 0.10025463,
       0.1000755 ])

## 3c

Преобразуйте получившийся вектор предсказаний `prediction` к значениям `{0,1}`. Это можно сделать, например, используя list comprehensions: https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions 

`predictionClass = [1 if prediction[i] > 0.5 else 0 for i in range(prediction.shape[0])]`

In [43]:
lims=0.2

In [44]:
# место для кода
predictionClass_LR = np.array([1 if y_pred_LR[i] > lims else 0 for i in range(y_pred_LR.shape[0])])

In [45]:
print(predictionClass_LR)

[0 0 0 ... 0 0 0]


In [46]:
# место для кода
predictionClass_Ridge = np.array([1 if y_pred_Ridge[i] > lims else 0 for i in range(y_pred_Ridge.shape[0])])

In [47]:
print(predictionClass_Ridge)

[0 0 0 ... 0 0 0]


In [49]:
# место для кода
predictionClass_Lasso = np.array([1 if y_pred_Lasso[i] > lims else 0 for i in range(y_pred_Lasso.shape[0])])

In [50]:
print(predictionClass_Lasso)

[0 0 0 ... 0 0 0]


In [51]:
# место для кода
predictionClass_SVM = np.array([1 if y_pred_SVM[i] > lims else 0 for i in range(y_pred_SVM.shape[0])])

In [52]:
predictionClass_SVM_L = np.array([1 if y_pred_SVM_L[i] > lims else 0 for i in range(y_pred_SVM_L.shape[0])])

In [53]:
print(predictionClass_SVM)

[0 0 0 ... 0 0 0]


In [54]:
print(predictionClass_SVM_L)

[0 0 0 ... 0 0 0]


## 3d

Постройте отчет по качеству классификации и матрицу ошибок. Как изменятся отчет и матрица ошибок, если изменить порог в задании 3c (по умолчанию его значение равно 0.5)?

*Функции, которые могут пригодиться при решении: `print(metrics.classification_report(...))`, `print(metrics.confusion_matrix(...))`*

Функция classification_report для каждого класса объектов считает точность (precision) в этом классе и полноту (recall). Полнота - это процент объектов данного класса, которые ваш метод предсказания тоже отнес к этому классу, среди всех объектов данного класса. Точность (precision) - то же самое, только среди всех объектов, предсказанных для этого класса.

Функция confusion_matrix возвращает матрицу с количествами объектов. Номера столбцов матрицы - это номера предсказанных классов, строки - это номера правильных классов. Например, элемент M[0,1] - это количество машин, где на самом деле IsBadBuy = 0, а вы предсказали 1.

In [55]:
print(np.count_nonzero(predictionClass_SVM))

974


In [56]:
print(np.count_nonzero(predictionClass_Lasso))

0


In [57]:
print(np.count_nonzero(predictionClass_LR))

2144


In [58]:
print(np.count_nonzero(predictionClass_SVM_L))

972


In [59]:
print(metrics.confusion_matrix(y_test,predictionClass_LR))

[[18050  1162]
 [ 1701   982]]


In [60]:
print(metrics.confusion_matrix(y_test,predictionClass_Ridge))

[[18045  1167]
 [ 1706   977]]


In [61]:
print(np.count_nonzero(predictionClass_Ridge))

2144


In [62]:
# место для кода
#predictionClass = np.array([1 if y_pred[i] > 0.1 else 0 for i in range(y_pred.shape[0])])
print(metrics.classification_report(y_test, predictionClass_LR))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93     19212
           1       0.46      0.37      0.41      2683

    accuracy                           0.87     21895
   macro avg       0.69      0.65      0.67     21895
weighted avg       0.86      0.87      0.86     21895



In [63]:
print(metrics.classification_report(y_test, predictionClass_Lasso))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93     19212
           1       0.00      0.00      0.00      2683

    accuracy                           0.88     21895
   macro avg       0.44      0.50      0.47     21895
weighted avg       0.77      0.88      0.82     21895



C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [64]:
print(metrics.classification_report(y_test, predictionClass_Ridge))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93     19212
           1       0.46      0.36      0.40      2683

    accuracy                           0.87     21895
   macro avg       0.68      0.65      0.67     21895
weighted avg       0.86      0.87      0.86     21895



In [65]:
print(metrics.classification_report(y_test, predictionClass_SVM))

              precision    recall  f1-score   support

           0       0.90      0.99      0.94     19212
           1       0.70      0.26      0.38      2683

    accuracy                           0.90     21895
   macro avg       0.80      0.62      0.66     21895
weighted avg       0.88      0.90      0.87     21895



In [66]:
print(metrics.classification_report(y_test, predictionClass_SVM_L))

              precision    recall  f1-score   support

           0       0.90      0.99      0.94     19212
           1       0.70      0.25      0.37      2683

    accuracy                           0.90     21895
   macro avg       0.80      0.62      0.66     21895
weighted avg       0.88      0.90      0.87     21895

